In [1]:
import numpy as np
import pandas as pd
import pydicom as dicom
import cv2
import os 
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
import matplotlib.pylab as plt
from tensorflow.keras.utils import Sequence
from keras.utils import data_utils
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import pandas as pd

In [4]:
train_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv')
test_df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
train_array = np.array(train_df['BraTS21ID'])
train_label =  np.array(train_df['MGMT_value'])
test_array = np.array(test_df['BraTS21ID'])
test_label =  np.array(test_df['MGMT_value'])
DIM = 512
NB_CHANNELS = 1
i = 0
BATCH_SIZE = 16 
a = 0
train_ds = pd.DataFrame(columns = ['path','label'] )
test_ds = pd.DataFrame(columns = ['path','label'] )

In [5]:
len(train_df)

585

In [6]:
test_df

,BraTS21ID,MGMT_value
0,1,0.5
1,13,0.5
2,15,0.5
3,27,0.5
4,37,0.5
...,...,...
82,826,0.5
83,829,0.5
84,833,0.5
85,997,0.5


In [7]:
print(str(10).zfill(5))

00010


In [33]:
dataframe_values = []
for a,b in zip(train_df['BraTS21ID'],train_df['MGMT_value']):
    folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{str(a).zfill(5)}')
    for folder in folders_list :  
        img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{str(a).zfill(5)}/{folder}')
        for img in img_list : 
            dataframe_values.append({'path' : f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{str(a).zfill(5)}/{folder}/{img}','label' : b})
train_ds  = pd.DataFrame.from_dict(dataframe_values)



In [34]:
test_ds = train_ds[300000:].reset_index()
train_ds = train_ds[:300000]

In [35]:
test_ds

,index,path,label
0,300000,../input/rsna-miccai-brain-tumor-radiogenomic-...,1
1,300001,../input/rsna-miccai-brain-tumor-radiogenomic-...,1
2,300002,../input/rsna-miccai-brain-tumor-radiogenomic-...,1
3,300003,../input/rsna-miccai-brain-tumor-radiogenomic-...,1
4,300004,../input/rsna-miccai-brain-tumor-radiogenomic-...,1
...,...,...,...
48636,348636,../input/rsna-miccai-brain-tumor-radiogenomic-...,0
48637,348637,../input/rsna-miccai-brain-tumor-radiogenomic-...,0
48638,348638,../input/rsna-miccai-brain-tumor-radiogenomic-...,0
48639,348639,../input/rsna-miccai-brain-tumor-radiogenomic-...,0


# unlabeled Data

In [9]:
dataframe_values = []
for a,b in zip(test_df['BraTS21ID'],test_df['MGMT_value']):
    folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(a).zfill(5)}')
    for folder in folders_list :    
        img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(a).zfill(5)}/{folder}')
        for img in img_list : 
            dataframe_values.append({'path' : f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{str(a).zfill(5)}/{folder}/{img}','label' : b})
test_ds  = pd.DataFrame.from_dict(dataframe_values)


In [10]:
test_ds

,path,label
0,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
1,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
2,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
3,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
4,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
...,...,...
51468,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
51469,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
51470,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5
51471,../input/rsna-miccai-brain-tumor-radiogenomic-...,0.5


In [11]:
train_augmentation_parameters = dict(
    rescale=1.0/255.0,
    rotation_range=10,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range = [0.8, 1.2])

valid_augmentation_parameters = dict(
    rescale=1.0/255.0)

# Using the training phase generators 
train_augmenter = ImageDataGenerator(**train_augmentation_parameters)
valid_augmenter = ImageDataGenerator(**valid_augmentation_parameters)

# in this part of code , i use keras dataGenerator  ,the problem here is that it generates the whole folder, so the first layer input for example is [900,512,512,1] which is going to give anOOM Error , so the solution here is to make a 16 (just an example ) FramesDatagenerator

In [12]:
class Frames_Generator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self,Sample_array, batch_size=1, dim=(512,512), shuffle=True,  train = True ):
        'Initialization'
        self.batch_size = batch_size
        self.Sample_array = Sample_array 
        self.shuffle = shuffle
        self.dim = dim
        self.train = train
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.Sample_array) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [str(self.Sample_array[k]).zfill(5) for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp, index)
        X =  np.array([X])[0]
        y =  np.array([y])
        y = y.reshape(y.shape[1])
        randomize = np.arange(y.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        #train_generator = train_augmenter.flow(X ,y,batch_size = 32 )
        return X, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.Sample_array))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp,index):
        label_train  = []
        Frame_train = [] 
        global a 
        global b
        print(list_IDs_temp[0])
        if self.train :
            folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{list_IDs_temp[0]}')
            for folder in folders_list : 
                img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{list_IDs_temp[0]}/{folder}')
                for img in img_list :
                    ds = dicom.dcmread(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/{list_IDs_temp[0]}/{folder}/{img}').pixel_array
                    img = cv2.resize(ds,self.dim)
                    img = img.reshape(self.dim[0],self.dim[1],1)
                    label_train.append(train_label[index])
                    Frame_train.append(img)
        else : 
            folders_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{list_IDs_temp[0]}')
            for folder in folders_list : 
                img_list = os.listdir(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{list_IDs_temp[0]}/{folder}')
                for img in img_list :
                    ds = dicom.dcmread(f'../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/{list_IDs_temp[0]}/{folder}/{img}').pixel_array
                    img = cv2.resize(ds,self.dim)
                    img = img.reshape(dim[0],dim[1],1)
                    label_train.append(test_label[index])
                    Frame_train.append(img)    
        return Frame_train, label_train

In [13]:
# Parameters
params = {'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train' : True }
valid_params = {'dim': (512,512),
          'batch_size': 1,
          'shuffle': True,
          'train':False}
training_generator = Frames_Generator(train_array, **params)
validation_generator = Frames_Generator(test_array, **valid_params)


# part2 

In [36]:
class Frames_Generator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self,Sample_df = np.arange(len(train_ds)//BATCH_SIZE-16) , batch_size=1, dim=(512,512), shuffle=True ,train_ds = train_ds,nb_steps = 0 ):
        'Initialization'
        self.batch_size = batch_size
        self.Sample_df = Sample_df
        self.shuffle = shuffle
        self.dim = dim
        self.train_ds = train_ds
        self.nb_steps = nb_steps
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.Sample_df) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [str(self.Sample_df[k]).zfill(5) for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        X =  np.array([X])[0]
        y =  np.array([y])
        y = y.reshape(y.shape[1])
        randomize = np.arange(y.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        #train_generator = train_augmenter.flow(X ,y,batch_size = 32 )
        return X, y
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.Sample_df))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        label_train  = []
        Frame_train = [] 
        i = 0
        while(i<16) : 
            ds = dicom.dcmread(self.train_ds['path'][self.nb_steps]).pixel_array
            img = cv2.resize(ds,self.dim)
            img = img.reshape(self.dim[0],self.dim[1],1)
            label_train.append(self.train_ds['label'][self.nb_steps])
            Frame_train.append(img)
            self.nb_steps += 1 
            i +=1
            if  self.nb_steps == len(self.train_ds) :
                self.nb_steps = 0
        return Frame_train, label_train

In [37]:
len(train_ds)//BATCH_SIZE-16

46

In [38]:
# Parameters
train_params = {'Sample_df':np.arange(len(train_ds)//BATCH_SIZE-16),
          'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train_ds' : train_ds,
         'nb_steps': 0}
test_params = {'Sample_df':np.arange(len(test_ds)//BATCH_SIZE-16),
          'dim': (512,512),
          'batch_size': 1,
          'shuffle': True, 
           'train_ds' : test_ds,
         'nb_steps': 0}
training_generator = Frames_Generator(**train_params)
validation_generator = Frames_Generator(**test_params)


In [93]:
x , y = validation_generator.__getitem__(0)

In [94]:
x[0].shape

(512, 512, 1)

In [95]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [58]:
arr = np.arange(len(train_ds)//BATCH_SIZE)
len(arr)

62

In [25]:
len(arr)*16

992

In [108]:

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same",input_shape = (DIM , DIM , NB_CHANNELS)))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 512, 512, 32)      320       
_________________________________________________________________
activation_35 (Activation)   (None, 512, 512, 32)      0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 512, 512, 32)      0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 512, 512, 32)      2048      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 170, 170, 32)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 170, 170, 64)      18496     
_________________________________________________________________
activation_36 (Activation)   (None, 170, 170, 64)     

In [109]:
checkpoint_path = "./cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer = opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(training_generator,epochs = 10,callbacks=[cp_callback])

In [62]:
arr_predicted = model.predict(x)

In [63]:
arr_predicted[arr_predicted<0.2]=0

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [1.        ],
       [0.99599767],
       [0.        ],
       [0.        ],
       [0.        ],
       [1.        ],
       [1.        ],
       [0.        ],
       [0.        ],
       [0.        ]], dtype=float32)